In [1]:

DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "opensanctions"

# Create driver
Also set the DB_NAME

In [39]:
import os
import pandas as pd
from graphdatascience import GraphDataScience # Python GDS client
import numpy as np
import threading
import multiprocessing

# And some cermony to create the driver and gds objects´
gds = GraphDataScience(DB_ULR, auth=(DB_USER, DB_PASS))
gds.set_database(DB_NAME)

In [12]:
gds.run_cypher(
    """ 
    CALL gds.graph.project('test','*',
        {
            HAS_IDENTIFIER: {orientation: 'UNDIRECTED'},
            ENTITY: {orientation: 'UNDIRECTED'},
            HAS_NAME: {orientation: 'UNDIRECTED'},
            ADDRESS_ENTITY: {orientation: 'UNDIRECTED'},
            UNKNOWN_LINK: {orientation: 'UNDIRECTED'},
            OWNERSHIP: {orientation: 'UNDIRECTED'},
            HAS_PHONE: {orientation: 'UNDIRECTED'},
            HAS_EMAIL: {orientation: 'UNDIRECTED'},
            OWNER: {orientation: 'UNDIRECTED'},
            DIRECTORSHIP: {orientation: 'UNDIRECTED'},
            REPRESENTATION: {orientation: 'UNDIRECTED'},
            HOLDER: {orientation: 'UNDIRECTED'},
            MEMBERSHIP: {orientation: 'UNDIRECTED'},
            ISSUER: {orientation: 'UNDIRECTED'},
            FAMILY: {orientation: 'UNDIRECTED'},
            ASSOCIATE: {orientation: 'UNDIRECTED'}
        }
    )
    """
)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'__ALL__': {'label': '*', 'properties': {}}}",{'HAS_IDENTIFIER': {'orientation': 'UNDIRECTED...,test,2409145,7361736,283


In [40]:
G = gds.graph.get('test');

In [71]:
politicians = gds.run_cypher("MATCH (p:Politician) return id(p) as id limit 300000")

In [72]:
politicians.head()

,id
0,3365985
1,3366214
2,3366282
3,3366356
4,3366460


In [42]:
nthreads = multiprocessing.cpu_count()
nthreads

10

In [51]:
chunks = np.array_split(politicians, nthreads)
all_threads = []

In [68]:
def worker(chunk, result, i):
    paths=gds.run_cypher(
        """ 
        MATCH (p:Politician) where id(p) in $ids
        with p
        call {
            with p
            CALL gds.allShortestPaths.dijkstra.stream('test', {
            sourceNode: p
            })
            YIELD index, targetNode, totalCost, path
            with index, totalCost, path, gds.util.asNode(targetNode) as target
            where target:SanctionedEntity
            RETURN
                index,
                target,
                totalCost,
                path
            LIMIT 1
        }
        return index,
                target,
                totalCost,
                nodes(path) as path
        """,
        params = { 'ids' : chunk['id'].to_list()  }
    )
    result[i] = paths

In [73]:

result = [None] * nthreads
for i in range(nthreads):
    thread = threading.Thread(target=worker, args=(chunks[i], result, i ))
    all_threads.append(thread)
    thread.start()

for thread in all_threads:
    thread.join()

In [74]:
result

[    index                                             target  totalCost  \
 0       0  (passportNumber, country, dissolutionDate, swi...        0.0   
 1       0  (passportNumber, country, dissolutionDate, swi...        0.0   
 2       0  (passportNumber, country, dissolutionDate, swi...        0.0   
 3       0  (passportNumber, country, dissolutionDate, swi...        0.0   
 4       0  (passportNumber, country, dissolutionDate, swi...        0.0   
 ..    ...                                                ...        ...   
 95      0  (passportNumber, country, dissolutionDate, swi...        0.0   
 96      0  (passportNumber, country, dissolutionDate, swi...        0.0   
 97      0  (passportNumber, country, dissolutionDate, swi...        0.0   
 98      0  (passportNumber, country, dissolutionDate, swi...        0.0   
 99      0  (passportNumber, country, dissolutionDate, swi...        0.0   
 
                                                  path  
 0   [(passportNumber, countr